In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

## Load data and rename columns

- Train: 29k rows x 10 cols, 'Beta' is outcome  
- Test: 20k rows... no outcome labels


In [23]:
train = pd.read_csv('data/train.csv')

# give the data names that don't suck
train = train.rename(columns={"Id": "id",
                              "CHR": "chromosome", 
                              "MAPINFO": "position",
                              "UCSC_CpG_Islands_Name": "island",  
                              "UCSC_RefGene_Group":"refgene",
                              "Relation_to_UCSC_CpG_Island": "rel_to_island",
                              "Regulatory_Feature_Group": "feature",
                              "Forward_Sequence":"fwd_seq",
                              "Beta": "outcome"})
train.shape

In [ ]:
## Same thing for test data
# test = pd.read_csv('data/test.csv')
# test = test.rename(columns={"Id": "id",
#                               "CHR": "chromosome", 
#                               "MAPINFO": "position",
#                               "UCSC_CpG_Islands_Name": "island",  
#                               "UCSC_RefGene_Group":"refgene",
#                               "Relation_to_UCSC_CpG_Island": "rel_to_island",
#                               "Regulatory_Feature_Group": "feature",
#                               "Forward_Sequence":"fwd_seq"})

In [24]:
train

,id,chromosome,position,island,refgene,rel_to_island,feature,fwd_seq,seq,outcome
0,cg00001583,1,200011786,chr1:200011400-200012055,Body;Body,Island,NaN,CGCCGTTGGGTAGCAGGAGGTAGAGCCTAGGCGAGAGCCTGGTGCG...,GCGCTTCTTTGCCCCGATGAGTTCGCCTCCCCAAACGCCTACTTCG...,0
1,cg00002719,1,169396706,chr1:169396621-169396869,TSS200,Island,NaN,GGACCACACTGCCATGGCAACAGCGTGCCTCTGCGTCCTCCATCCG...,AGTAAGAGACGGAAATAAATTCCTTCCTCCCTGAGTGTCTGGTAAA...,0
2,cg00003202,1,151319512,chr1:151319326-151319545,5'UTR;5'UTR,Island,Promoter_Associated,TGCAAAGTGGAGCGGGGGTGGGGCGGGGTAGCGGAACTCATGCGCT...,AAAGGACTGAAATGCCCAGCAGGTGCTCAAGAATTGCTACCATGGC...,0
3,cg00007036,1,33742089,chr1:33741774-33742381,Body,Island,NaN,GTCGGCCTCGTCGCAGCAGCCGTTGCTAGTGCCGCCGGCACCCGCC...,TTCTTGAAGATAACTTTCCAGAAGTACAATTTCTTGAGTCAAAGGG...,1
4,cg00007898,1,205180627,chr1:205179963-205180910,1stExon;1stExon,Island,Promoter_Associated,GGCCGAAGCCCCGGCACAGCTCGCGGATCATTCCGCCGCCGCCGGG...,AAAGGAAGCAAGACGTTAGGAAAGATTAAAGACTGCACGATTTTAA...,0
...,...,...,...,...,...,...,...,...,...,...
29060,ch.10.2633792F,10,123987885,NaN,Body;Body;Body;Body,NaN,NaN,AAGCAGTCCTCTGCCGCTTCGTCCTTGATGGCAGAGGGCAAGGGAC...,AAGAGACTCTGCAGGCTGCCCAGGGTGGTGAGAACAATGATGATGG...,0
29061,ch.10.125213076F,10,125223086,NaN,NaN,NaN,NaN,AGCAGAGCAGGAGTGGGATTCTGAAACCCTTGCTCTTCTGGTTATT...,TTACTCAGACGCTGCAGGTTGTGAATTTCAGTCCTGGTAAATCATG...,0
29062,ch.10.2809628F,10,130365546,NaN,NaN,NaN,NaN,ATGCTTAGTACTTCATTAAGACATTTGTGAGCAAACTCATTCCCAT...,CTCCAAGCCTGAAGAGCCTCTGCCTTCCCTTCCCTCACTCTGCGTG...,0
29063,ch.10.130844938F,10,130954948,NaN,NaN,NaN,NaN,GATCGTCCAAGGTGTCTCTTAGTATGATCATGCCTTATGATTAAAG...,TCATGGCGGCAGGGATGGAAGCTATGCATGGGTTCAGCAACATGGA...,0


In [5]:
train.describe(include='all')

,Id,CHR,MAPINFO,UCSC_CpG_Islands_Name,UCSC_RefGene_Group,Relation_to_UCSC_CpG_Island,Regulatory_Feature_Group,Forward_Sequence,seq,Beta
count,29065,29065.000000,2.906500e+04,23717,24822,23717,16421,29065,29065,29065.000000
unique,29065,NaN,NaN,9047,1088,5,8,29065,29065,NaN
top,cg17474323,NaN,NaN,chr6:32939275-32940954,Body,Island,Promoter_Associated,TCTGGTTTAGCTAACGGGTGGGTGGAAGGGAACCACTCTCAGAGCG...,AGGCGAAGAAAAGGTTGCCCCAGTTATGCGAAGAGCTAGTCCCGTT...,NaN
freq,1,NaN,NaN,31,4804,18269,11797,1,1,NaN
mean,NaN,4.781283,8.819196e+07,NaN,NaN,NaN,NaN,NaN,NaN,0.303527
std,NaN,2.920974,6.707768e+07,NaN,NaN,NaN,NaN,NaN,NaN,0.459789
min,NaN,1.000000,4.604300e+04,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,NaN,2.000000,2.972100e+07,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
50%,NaN,5.000000,7.976474e+07,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
75%,NaN,7.000000,1.400985e+08,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
